# Preparing data

In [3]:
import pandas as pd

In [2]:
# path_dir = "/ist/ist-share/scads/can/fame_fake_data/data"

# """
# multinli_1.0_dev_matched_add_tokens.jsonl
# multinli_1.0_dev_matcheddev_anti_bias_add_tokens.jsonl
# multinli_1.0_dev_matched.jsonl`a
# multinli_1.0_dev_mismatched.jsonl
# multinli_1.0_train_add_tokens.jsonl
# multinli_1.0_train.jsonl

# """

# train_df = pd.read_csv("train_nli_features.csv")
# dev_df = pd.read_csv("dev_match_nli_features.csv")
# test_df = pd.read_csv("dev_mismatch_nli_features.csv")
# hans_df = pd.read_csv("hans_features.csv")
# train_df.shape, dev_df.shape, test_df.shape, hans_df.shape

# train_df.head()

'\nmultinli_1.0_dev_matched_add_tokens.jsonl\nmultinli_1.0_dev_matcheddev_anti_bias_add_tokens.jsonl\nmultinli_1.0_dev_matched.jsonl\nmultinli_1.0_dev_mismatched.jsonl\nmultinli_1.0_train_add_tokens.jsonl\nmultinli_1.0_train.jsonl\n\n'

In [ ]:
train_df = pd.read_csv("train_nli_features.csv")
dev_df = pd.read_csv("dev_match_nli_features.csv")
test_df = pd.read_csv("dev_mismatch_nli_features.csv")
train_df.shape, dev_df.shape, test_df.shape

train_df.head()

In [4]:
new_train = pd.DataFrame()
new_train["pairID"] = train_df["pairID"]
new_train["label"] = train_df["gold_label"]
new_train["sub"] = train_df["subsequence"]
new_train["cons"] = train_df["constituent"]
# new_train["lexical_overlap"] = train_df["lexical_overlap"]
new_train["overlapping score"] = train_df["overlapping score"]

new_dev = pd.DataFrame()
new_dev["pairID"] = dev_df["pairID"]
new_dev["label"] = dev_df["gold_label"]
new_dev["sub"] = dev_df["subsequence"]
new_dev["cons"] = dev_df["constituent"]
# new_dev["lexical_overlap"] = dev_df["lexical_overlap"]
new_dev["overlapping score"] = dev_df["overlapping score"]

new_test = pd.DataFrame()
new_test["pairID"] = test_df["pairID"]
new_test["label"] = test_df["gold_label"]
new_test["sub"] = test_df["subsequence"]
new_test["cons"] = test_df["constituent"]
# new_test["lexical_overlap"] = test_df["lexical_overlap"]
new_test["overlapping score"] = test_df["overlapping score"]

new_hans = pd.DataFrame()
new_hans["pairID"] = hans_df["pairID"]
new_hans["label"] = hans_df["gold_label"]
new_hans["sub"] = hans_df["subsequence"]
new_hans["cons"] = hans_df["constituent"]
# new_hans["lexical_overlap"] = hans_df["lexical_overlap"]
new_hans["overlapping score"] = hans_df["overlapping score"]

new_train.shape, new_dev.shape, new_test.shape, new_hans.shape

((392702, 5), (10000, 5), (10000, 5), (30000, 5))

In [5]:
new_train.head()

,pairID,label,sub,cons,overlapping score
0,31193n,neutral,-,-,0.444444
1,101457e,entailment,-,-,0.666667
2,134793e,entailment,-,-,0.250000
3,37397e,entailment,-,-,0.400000
4,50563n,neutral,-,-,0.625000


In [6]:
# # 2 class {entailment: 1, contrandiction: 0, neatral: 0}
# def relabel(label):
#     if label == 'contradiction' or label == 'neutral':
#         return 0
#     else:
#         return 1

# 3 class {entailment: 0, contrandiction: 1, neatral: 2}
def relabel(label):
    if label == 'contradiction':
        return 1
    elif label == 'neutral':
        return 2
    else:
        return 0

# {-: 0, easy: 1, hard: 1}
def refeatures(feature):
    if feature == 'easy' and feature == 'hard':
        return 1
    else:
        return 0

In [7]:
new_train["label"] = new_train["label"].apply(relabel)
# new_train["lexical_overlap"] = new_train["lexical_overlap"].apply(refeatures)
new_train["sub"] = new_train["sub"].apply(refeatures)
new_train["cons"] = new_train["cons"].apply(refeatures)

new_dev["label"] = new_dev["label"].apply(relabel)
# new_dev["lexical_overlap"] = new_dev["lexical_overlap"].apply(refeatures)
new_dev["sub"] = new_dev["sub"].apply(refeatures)
new_dev["cons"] = new_dev["cons"].apply(refeatures)

new_test["label"] = new_test["label"].apply(relabel)
# new_test["lexical_overlap"] = new_test["lexical_overlap"].apply(refeatures)
new_test["sub"] = new_test["sub"].apply(refeatures)
new_test["cons"] = new_test["cons"].apply(refeatures)

# new_hans["label"] = new_hans["label"].apply(relabel)
# new_hans["lexical_overlap"] = new_hans["lexical_overlap"].apply(refeatures)
# new_hans["sub"] = new_hans["sub"].apply(refeatures)
# new_hans["cons"] = new_hans["cons"].apply(refeatures)

new_train.head()

,pairID,label,sub,cons,overlapping score
0,31193n,2,0,0,0.444444
1,101457e,0,0,0,0.666667
2,134793e,0,0,0,0.250000
3,37397e,0,0,0,0.400000
4,50563n,2,0,0,0.625000


# Train Model

In [8]:
from __future__ import division
import pandas as pd
import numpy as np
from sklearn import preprocessing, svm, metrics, tree, decomposition, svm
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier, OrthogonalMatchingPursuit
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import *
from sklearn.preprocessing import StandardScaler
import random
import matplotlib.pyplot as plt
from scipy import optimize
import time
import seaborn as sns

import sklearn
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base

In [9]:
def define_clfs_params(grid_size):
    """Define defaults for different classifiers.
    Define three types of grids:
    Test: for testing your code
    Small: small grid
    Large: Larger grid that has a lot more parameter sweeps
    """

    clfs = {'RF': RandomForestClassifier(n_estimators=50, n_jobs=-1),
        'ET': ExtraTreesClassifier(n_estimators=10, n_jobs=-1, criterion='entropy'),
        'AB': AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), algorithm="SAMME", n_estimators=200),
        'LR': LogisticRegression(penalty='l1', C=1e5),
        'SVM': svm.SVC(kernel='linear', probability=True, random_state=0),
        'GB': GradientBoostingClassifier(learning_rate=0.05, subsample=0.5, max_depth=6, n_estimators=10),
        'NB': GaussianNB(),
        'DT': DecisionTreeClassifier(),
        'SGD': SGDClassifier(loss="hinge", penalty="l2"),
        'KNN': KNeighborsClassifier(n_neighbors=3) 
            }

    large_grid = {'RF':{'n_estimators': [1,10,100,1000,10000], 'max_depth': [1,5,10,20,50,100], 'max_features': ['sqrt','log2'],'min_samples_split': [2,5,10], 'n_jobs': [-1]},
        'LR': {'penalty': ['l2'], 'C': [0.00001,0.0001,0.001,0.01,0.1,1,10], 'class_weight': ['balanced', None]},
        'SGD': { 'loss': ['hinge','log','perceptron'], 'penalty': ['l2','l1','elasticnet']},
        'ET': { 'n_estimators': [1,10,100,1000,10000], 'criterion' : ['gini', 'entropy'] ,'max_depth': [1,5,10,20,50,100], 'max_features': ['sqrt','log2'],'min_samples_split': [2,5,10], 'n_jobs': [-1]},
        'AB': { 'algorithm': ['SAMME', 'SAMME.R'], 'n_estimators': [1,10,100,1000,10000]},
        'GB': {'n_estimators': [1,10,100,1000,10000], 'learning_rate' : [0.001,0.01,0.05,0.1,0.5],'subsample' : [0.1,0.5,1.0], 'max_depth': [1,3,5,10,20,50,100]},
        'NB' : {},
        'DT': {'criterion': ['gini', 'entropy'], 'max_depth': [1,5,10,20,50,100],'min_samples_split': [2,5,10]},
        'SVM' :{'C' :[0.00001,0.0001,0.001,0.01,0.1,1,10],'kernel':['linear']},
        'KNN' :{'n_neighbors': [1,5,10,25,50,100],'weights': ['uniform','distance'],'algorithm': ['auto','ball_tree','kd_tree']}
           }
    
    small_grid = {'RF':{'n_estimators': [10,100], 'max_depth': [5,50], 'max_features': ['sqrt','log2'],'min_samples_split': [2,10], 'n_jobs': [-1]}, 
    'LR': {'penalty': ['l2'], 'C': [0.00001,0.001,0.1,1,10], 'class_weight': ['balanced', None]},
    'SGD': {'loss': ['hinge','log','perceptron'], 'penalty': ['l2','l1','elasticnet']},
    'ET': {'n_estimators': [10,100], 'criterion' : ['gini', 'entropy'] ,'max_depth': [5,50], 'max_features': ['sqrt','log2'],'min_samples_split': [2,10], 'n_jobs': [-1]},
    'AB': {'algorithm': ['SAMME', 'SAMME.R'], 'n_estimators': [1,10,100,1000,10000]},
    'GB': {'n_estimators': [10,100], 'learning_rate' : [0.001,0.1,0.5],'subsample' : [0.1,0.5,1.0], 'max_depth': [5,50]},
    'NB' : {},
    'DT': {'criterion': ['gini', 'entropy'], 'max_depth': [1,5,10,20,50,100],'min_samples_split': [2,5,10]},
    'SVM' :{'C' :[0.00001,0.0001,0.001,0.01,0.1,1,10],'kernel':['linear']},
    'KNN' :{'n_neighbors': [1,5,10,25,50,100],'weights': ['uniform','distance'],'algorithm': ['auto','ball_tree','kd_tree']}
           }

    gam_grid = {
        'LR': {'penalty': ['l2'], 'C': [0.01], 'class_weight': ['balanced']},
        'SGD': { 'loss': ['hinge','log','perceptron'], 'penalty': ['l2','l1','elasticnet']},
        'AB': { 'algorithm': ['SAMME', 'SAMME.R'], 'n_estimators': [1,10,100,1000,10000]},
        'GB': {'n_estimators': [1,10,100,1000,10000], 'learning_rate' : [0.001,0.01,0.05,0.1,0.5],'subsample' : [0.1,0.5,1.0], 'max_depth': [1,3,5,10,20,50,100]},
        'NB' : {},
        'DT': {'criterion': ['gini', 'entropy'], 'max_depth': [1,5,10,20,50,100],'min_samples_split': [2,5,10]},
        'SVM' :{'C' :[0.00001,0.0001,0.001,0.01,0.1,1,10],'kernel':['linear']},
           }
    
    test_grid = {'RF':{'n_estimators': [1], 'max_depth': [1], 'max_features': ['sqrt'],'min_samples_split': [10]}, 
    'LR': {'penalty': ['l2'], 'C': [0.01], 'class_weight': ['balanced', None]},
    'SGD': {'loss': ['perceptron'], 'penalty': ['l1', 'elasticnet']},
    'ET': {'n_estimators': [1], 'criterion' : ['gini'] ,'max_depth': [1], 'max_features': ['sqrt'],'min_samples_split': [10]},
    'AB': {'algorithm': ['SAMME'], 'n_estimators': [1]},
    'GB': {'n_estimators': [1], 'learning_rate' : [0.1],'subsample' : [0.5], 'max_depth': [1]},
    'NB' : {},
    'DT': {'criterion': ['gini'], 'max_depth': [1],'min_samples_split': [10]},
    'SVM' :{'C' :[0.01],'kernel':['linear']},
    'KNN' :{'n_neighbors': [5], 'weights': ['uniform'], 'algorithm': ['auto']}
           }
    
    if (grid_size == 'gam'):
        return clfs, gam_grid
    elif (grid_size == 'small'):
        return clfs, small_grid
    elif (grid_size == 'test'):
        return clfs, test_grid
    else:
        return 0, 0

In [10]:
# select features to use
X_train = new_train.iloc[:, 2:].values
X_dev = new_dev.iloc[:, 2:].values
X_test = new_test.iloc[:, 2:].values
X_hans = new_hans.iloc[:, 2:].values

# define label
y_train = new_train.iloc[:, 1].values
y_dev = new_dev.iloc[:, 1].values
y_test = new_test.iloc[:, 1].values
y_hans = new_hans.iloc[:, 1].values

In [11]:
# NOTEBOOK = 0

# def clf_loop(models_to_run, clfs, grid, X_train, X_test, y_train, y_test):
#     """Runs the loop using models_to_run, clfs, gridm and the data
#     """
#     results_df =  pd.DataFrame(columns=('model_type', 'clf', 'parameters', 'auc-roc', 'acc', 'classification report', 'confusion matrix'))

#     for n in range(1, 2):
#         # create training and valdation sets
#         # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
#         for index, clf in enumerate([clfs[x] for x in models_to_run]):
#             print(models_to_run[index])
#             parameter_values = grid[models_to_run[index]]
#             for p in ParameterGrid(parameter_values):
#                 try:
#                     clf.set_params(**p)
#                     y_pred_probs = clf.fit(X_train, y_train).predict(X_test) # HERE
#                     roc_y_pred_probs = clf.fit(X_train, y_train).predict_proba(X_test)[:,1]
#                     confusion = pd.DataFrame(confusion_matrix(y_test, y_pred_probs, labels = [0, 1]), index = [0, 1], columns = [0, 1])


#                     # you can also store the model, feature importances, and prediction scores
#                     # we're only storing the metrics for now
#                     y_pred_probs_sorted, y_test_sorted = zip(*sorted(zip(y_pred_probs, y_test), reverse=True))
#                     results_df.loc[len(results_df)] = [models_to_run[index], clf, p,
#                                                        roc_auc_score(y_test, roc_y_pred_probs),
#                                                        sklearn.metrics.accuracy_score(y_test, y_pred_probs),
#                                                        classification_report(y_test, y_pred_probs, output_dict=True),
#                                                        confusion.to_dict(orient="list")]

#                     #print(results_df)
#                     if NOTEBOOK == 1:
#                         plot_precision_recall_n(y_test, y_pred_probs, clf)
#                 except IndexError as e:
#                     print('Error:', e)
#                     continue

#     return results_df

NOTEBOOK = 0

def clf_loop(models_to_run, clfs, grid, X_train, X_test, y_train, y_test):
    """Runs the loop using models_to_run, clfs, gridm and the data
    """
    results_df =  pd.DataFrame(columns=('model_type', 'clf', 'parameters', 'auc-roc', 'acc', 'classification report', 'confusion matrix'))

    for n in range(1, 2):
        # create training and valdation sets
        # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
        for index, clf in enumerate([clfs[x] for x in models_to_run]):
            print(models_to_run[index])
            parameter_values = grid[models_to_run[index]]
            for p in ParameterGrid(parameter_values):
                try:
                    clf.set_params(**p)
                    y_pred_probs = clf.fit(X_train, y_train).predict(X_test) # HERE
                    roc_y_pred_probs = clf.fit(X_train, y_train).predict_proba(X_test)[:,1]
                    confusion = pd.DataFrame(confusion_matrix(y_test, y_pred_probs, labels = [0, 1, 2]), index = [0, 1, 2], columns = [0, 1, 2])


                    # you can also store the model, feature importances, and prediction scores
                    # we're only storing the metrics for now
                    y_pred_probs_sorted, y_test_sorted = zip(*sorted(zip(y_pred_probs, y_test), reverse=True))
                    results_df.loc[len(results_df)] = [models_to_run[index], clf, p,
                                                       roc_auc_score(y_test, roc_y_pred_probs, multi_class="ovr"),
                                                       sklearn.metrics.accuracy_score(y_test, y_pred_probs),
                                                       classification_report(y_test, y_pred_probs, output_dict=True),
                                                       confusion.to_dict(orient="list")]

                    #print(results_df)
                    if NOTEBOOK == 1:
                        plot_precision_recall_n(y_test, y_pred_probs, clf)
                except IndexError as e:
                    print('Error:', e)
                    continue

    return results_df

In [12]:
def main():

    # define grid to use: test, small, large
    grid_size = 'gam'
    clfs, grid = define_clfs_params(grid_size)

    # define models to run
    models_to_run=['LR']

    # call clf_loop and store results in results_df
    results_df = clf_loop(models_to_run, clfs, grid, X_train, X_dev, y_train, y_dev)
    print(results_df)
    if NOTEBOOK == 1:
        results_df


if __name__ == '__main__':
    main()

LR
Error: axis 1 is out of bounds for array of dimension 1
Empty DataFrame
Columns: [model_type, clf, parameters, auc-roc, acc, classification report, confusion matrix]
Index: []


In [13]:
logmodel = LogisticRegression(penalty='l2', C=0.01, class_weight='balanced')
t = logmodel.fit(X_train, y_train)

In [14]:
# import pickle
# modelname = 'mnli_lr_model.sav'
# pickle.dump(t, open(modelname, 'wb'))

In [15]:
# counterfactual=np.array([[0,0,1.0]])

In [ ]:
# modelname = 'mnli_lr_model.sav'
# loaded_model = pickle.load(open(modelname, 'rb'))
# loaded_model.predict_proba(counterfactual)

In [23]:
# loaded_model.predict_proba(counterfactual)

In [ ]:
# X_train[0]

In [ ]:
# np.mean(X_train, axis=0)

In [ ]:
# t.predict_proba(counterfactual)

In [17]:
pred_train = t.predict_proba(X_train).tolist()
pred_dev = t.predict_proba(X_dev).tolist()
pred_test = t.predict_proba(X_test).tolist()
pred_hans = t.predict_proba(X_hans).tolist()

In [18]:
new_train_df = pd.DataFrame()
new_train_df["gold_label"] = train_df["gold_label"]
new_train_df["sentence1"] = train_df["sentence1"]
new_train_df["sentence2"] = train_df["sentence2"]
new_train_df["bias_probs"] = pred_train

new_dev_df = pd.DataFrame()
new_dev_df["gold_label"] = dev_df["gold_label"]
new_dev_df["sentence1"] = dev_df["sentence1"]
new_dev_df["sentence2"] = dev_df["sentence2"]
new_dev_df["bias_probs"] = pred_dev

new_test_df = pd.DataFrame()
new_test_df["gold_label"] = test_df["gold_label"]
new_test_df["sentence1"] = test_df["sentence1"]
new_test_df["sentence2"] = test_df["sentence2"]
new_test_df["bias_probs"] = pred_test

new_hans_df = pd.DataFrame()
new_hans_df["gold_label"] = hans_df["gold_label"]
new_hans_df["sentence1"] = hans_df["premise"]
new_hans_df["sentence2"] = hans_df["hypothesis"]
new_hans_df["bias_probs"] = pred_hans

new_train_df.shape, new_dev_df.shape, new_test_df.shape, new_hans_df.shape

((392702, 4), (10000, 4), (10000, 4), (30000, 4))

In [19]:
new_test_df.isnull().sentence1.values.sum()

0

In [20]:
# replace nall to n\/a
new_train_df.sentence2 = new_train_df.sentence2.fillna('n\/a')

In [36]:
new_train_df.head()

,gold_label,sentence1,sentence2,bias_probs
0,neutral,Conceptually cream skimming has two basic dime...,Product and geography are what make cream skim...,"[0.33998758770118354, 0.34308485129272914, 0.3..."
1,entailment,you know during the season and i guess at at y...,You lose the things to the following level if ...,"[0.5033300255414216, 0.2859676470184237, 0.210..."
2,entailment,One of our number will carry out your instruct...,A member of my team will execute your orders w...,"[0.21996300239532512, 0.36692578681271204, 0.4..."
3,entailment,How do you know? All this is their information...,This information belongs to them.,"[0.30998086898840815, 0.35088778924181063, 0.3..."
4,neutral,yeah i tell you what though if you go price so...,The tennis shoes have a range of prices.,"[0.4718813548036844, 0.29858867684924567, 0.22..."


In [22]:
import json

train_json = new_train_df.to_json(orient='records', lines=True)
with open('train_prob_korn_lr_overlapping_sample_weight_3class.jsonl', 'w') as json_file:
    json_file.write(train_json)

dev_json = new_dev_df.to_json(orient='records', lines=True)    
with open('dev_prob_korn_lr_overlapping_sample_weight_3class.jsonl', 'w') as json_file:
    json_file.write(dev_json)

test_json = new_test_df.to_json(orient='records', lines=True)     
with open('test_prob_korn_lr_overlapping_sample_weight_3class.jsonl', 'w') as json_file:
    json_file.write(test_json)
    
hans_json = new_hans_df.to_json(orient='records', lines=True)     
with open('hans_prob_korn_lr_overlapping_sample_weight_3class.jsonl', 'w') as json_file:
    json_file.write(hans_json)

In [38]:
predictions = t.predict(X_train)
predict_prob = t.predict_proba(X_train)[:,1]
new_train['prediction'] = predictions
new_train['prob_score'] = predict_prob
new_train.head()

,pairID,label,sub,cons,lexical_overlap,overlapping score,prediction,prob_score,weight_score,trimmed_weight_score
0,31193n,2,0,0,0,0.444444,1,0.343085,0.431538,0.431538
1,101457e,0,0,0,0,0.666667,0,0.285968,0.509842,0.509842
2,134793e,0,0,0,0,0.250000,2,0.366926,0.400783,0.400783
3,37397e,0,0,0,0,0.400000,1,0.350888,0.421347,0.421347
4,50563n,2,0,0,0,0.625000,0,0.298589,0.491978,0.491978


In [39]:
# # just for checking 2 class
# score = {'1': 0, '0': 0}
# for i in range(len(new_train['label'])):
#     if new_train['label'][i] == 1:
#         score['1'] += 1
#     else:
#         score['0'] += 1

# just for checking 3 class
score = {'2': 0, '1': 0, '0': 0}
for i in range(len(new_train['label'])):
    if new_train['label'][i] == 2:
        score['2'] += 1
    elif new_train['label'][i] == 1:
        score['1'] += 1
    else:
        score['0'] += 1
        
score

{'2': 130900, '1': 130903, '0': 130899}

In [40]:
confusion = confusion_matrix(new_train['label'], new_train['prediction'])
confusion

array([[79717, 16611, 34571],
       [49593, 19884, 61426],
       [37563, 19929, 73408]])

In [41]:
count = 0
for i in range(len(new_train['label'])):
    if new_train['label'][i] != new_train['prediction'][i]:
        count += 1
count

219693

In [42]:
all_scores = []
for i in range(len(new_train['prob_score'])):
    all_scores.append(new_train['prob_score'][i])

In [43]:
max(all_scores), min(all_scores) 

(0.3691658644713939, 0.17571198816068542)

In [62]:
# ps
def give_weight(label, prob_score):
    result = 1 / prob_score
    return result

# # clark
# def give_weight(label, prob_score):
#     result = 1 - prob_score
#     return result

# # focal
# def give_weight(label, prob_score):
#     result = (1 - prob_score)**2
#     return result

In [63]:
new_train['weight_score'] = new_train[['label', 'prob_score']].apply(lambda x: give_weight(*x), axis=1)
new_train.head()

,pairID,label,sub,cons,lexical_overlap,overlapping score,prediction,prob_score,weight_score,trimmed_weight_score
0,31193n,2,0,0,0,0.444444,1,0.343085,2.914731,0.656915
1,101457e,0,0,0,0,0.666667,0,0.285968,3.496899,0.714032
2,134793e,0,0,0,0,0.250000,2,0.366926,2.725347,0.633074
3,37397e,0,0,0,0,0.400000,1,0.350888,2.849914,0.649112
4,50563n,2,0,0,0,0.625000,0,0.298589,3.349089,0.701411


In [64]:
train_weight = []
for i in range(len(new_train['weight_score'])):
    train_weight.append(new_train['weight_score'][i])

In [65]:
max(train_weight), min(train_weight)

(5.69113132500395, 2.708809497952616)

In [66]:
new_train.quantile(q=0.95)

label                   2.000000
sub                     0.000000
cons                    0.000000
lexical_overlap         0.000000
overlapping score       0.800000
prediction              2.000000
prob_score              0.369002
weight_score            4.127717
trimmed_weight_score    0.757735
Name: 0.95, dtype: float64

In [67]:
w_score = new_train.quantile(q=0.95).weight_score

In [68]:
def trim(weight_score):
    if weight_score >= w_score:
        return w_score
    else: 
        return weight_score

In [69]:
new_train['trimmed_weight_score'] = new_train['weight_score'].apply(trim)
new_train.head()

,pairID,label,sub,cons,lexical_overlap,overlapping score,prediction,prob_score,weight_score,trimmed_weight_score
0,31193n,2,0,0,0,0.444444,1,0.343085,2.914731,2.914731
1,101457e,0,0,0,0,0.666667,0,0.285968,3.496899,3.496899
2,134793e,0,0,0,0,0.250000,2,0.366926,2.725347,2.725347
3,37397e,0,0,0,0,0.400000,1,0.350888,2.849914,2.849914
4,50563n,2,0,0,0,0.625000,0,0.298589,3.349089,3.349089


In [70]:
new_train.to_csv("train_overlapping_feature_weight_ps3class.csv")